In [10]:
import cobra
from cobra.io import load_json_model
import numpy as np
import pandas as pd
cobra_model = load_json_model("./COBRA_models/GEM_Recon2_thermocurated_redHUMAN.json")
import itertools
def grow_dict(previo, nuevo):  
    """Esto es un .update(overwritte=False)?"""

    not_new_keys = set(previo.keys()).intersection(set(nuevo.keys()))
    new_keys     = set(nuevo.keys()).difference(set(previo.keys()))
    for new in new_keys:
        previo[new] =np.unique(nuevo[new]).tolist() 
    for common in not_new_keys:
        previo[common].extend(np.unique(nuevo[common]).tolist() )
    return previo
# Sacamos los genes de Recon hacia un diccionario
# TODO: convertir est en una horrible dict_comprehension { curse : cursed for c in curseds }
dict_rxn_to_gene = dict()
genes            = [g.id for g in cobra_model.genes]

#print(genes)


for id in genes:
    # fmt: off
    rxns             = [r.id for r in cobra_model.genes.get_by_id(id)._reaction] # reacciones asociadas al gen de la iter
    updated_dict  : dict[str, list[str]] = dict(zip(rxns,itertools.repeat([id]))) # zipping con la { reaccion : [id_gen] }
    dict_rxn_to_gene = grow_dict(dict_rxn_to_gene, updated_dict) # merge recursivo

In [11]:
edges_node_names_explainer_subgraphs = pd.read_csv("./results/data/edges_node_names_explainer_subgraphs.csv")
genes = []


for node_1, node_2 in zip(edges_node_names_explainer_subgraphs.node1.tolist(), edges_node_names_explainer_subgraphs.node2.tolist()):   
    
    
    if node_2 in dict_rxn_to_gene.keys():
        
        genes.append(dict_rxn_to_gene[node_2])
        
    elif node_1 in dict_rxn_to_gene.keys():
        
        genes.append(dict_rxn_to_gene[node_1])
        
    else:
        genes.append([''])
        


results_for_R =  pd.concat([pd.Series(genes, name='genes'), edges_node_names_explainer_subgraphs], axis=1).drop('Unnamed: 0',axis=1).reset_index(drop=True)


In [14]:
results_for_R[results_for_R['0']> 0].to_csv("./results/data/results_for_R_non_zero.csv")

In [13]:
results_for_R.to_csv("./results/data/results_for_R.csv")